In [43]:
import glob
import os
import pandas as pd
from sklearn import model_selection
from PIL import Image
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import albumentations as aug
import efficientnet_pytorch
import random
import torchvision
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [44]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5          
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False
p=0.5
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

TRAIN_FILE = 'D:\\Dataset\\Cassava Competiton\\train.csv'
TEST_FILE = 'D:\\Dataset\\Cassava Competiton\\test_images\\'
MODEL_PATH = 'D:\\Dataset\\Cassava Competiton\\'

In [45]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [70]:
train_df = pd.read_csv(TRAIN_FILE)
test_images = [i for i in glob.glob(f'{TEST_FILE}\*')]
test_images_sub = [os.path.basename(i) for i in glob.glob(f'{TEST_FILE}\*')]

In [47]:
train_df.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [48]:
if __name__ == "__main__":
    input_path='D:\\Dataset\\Cassava Competiton\\'
    train_df["fold"] = -1
    train_df = train_df.sample(frac=1).reset_index(drop=True)
    y = train_df.label.values
    skf = model_selection.StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    for fold,(idxT,idxV) in enumerate(skf.split(X=train_df, y=y)):
        train_df.loc[idxV, "fold"] = fold
    print(train_df.label.value_counts())
    train_df.to_csv(os.path.join(input_path, "train_folds.csv"), index=False)

3    13158
4     2577
2     2386
1     2189
0     1087
Name: label, dtype: int64


In [49]:
train_df.label.value_counts()

3    13158
4     2577
2     2386
1     2189
0     1087
Name: label, dtype: int64

In [50]:
train_df['Prediction'] = -1
test = pd.DataFrame()
test['image_id'] = ['2216849948.jpg']
test['Prediction'] = -1

In [51]:
class Cassava_Train_DS:
    
    def __init__(self, image_paths, targets, resize, augmentations=None):
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item])
        targets = self.targets[item]
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )
        image = np.array(image)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        dct = {
            "image": torch.tensor(image),
            "targets": torch.tensor(targets),
        }
                
        return dct
    
class Cassava_Test_DS:
    
    def __init__(self, image_paths, resize, augmentations=None):
        self.image_paths = image_paths
        self.resize = resize
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item])
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )
        image = np.array(image)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        dct = {
            "image": torch.tensor(image),
        }
        
        return dct

In [52]:
class CassavaModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.backbone = torchvision.models.resnet18(pretrained=True)
        
        in_features = self.backbone.fc.in_features

        self.out = nn.Linear(in_features, 5)
        
    def forward(self, image):
        batch_size, C, H, W = image.shape
        
        x = self.backbone.conv1(image)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)

        x = self.out(x)

            
        return x

In [53]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['image'].to(device), data['targets'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        #print(data['image'])
        inputs, targets = data['image'].to(device), data['targets'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    #print(valid_preds.shape)
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['image'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)

    return preds

In [54]:
train_aug = aug.Compose(
        [
            
        aug.RandomResizedCrop(226, 226),
        aug.Transpose(p=p),
        aug.HorizontalFlip(p=p),
        aug.VerticalFlip(p=p),
        aug.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),  
        ]
    )

valid_aug = aug.Compose(
       [
            
        aug.CenterCrop(226, 226, p=1.0),
        aug.Resize(226, 226),
        aug.Transpose(p=p),
        aug.VerticalFlip(p=p),
        aug.HorizontalFlip(p=p),
        aug.VerticalFlip(p=p),
        aug.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),         ]
    )

In [55]:
def run_training(fold, seed):
    training_data_path = 'D:\\Dataset\\Cassava Competiton\\train_images\\'
    
    seed_everything(seed)
    
    resize = (226,226)
    
    train = pd.read_csv(input_path+'train_folds.csv')
    
                  
    trn_idx = train[train['fold'] != fold].index
    val_idx = train[train['fold'] == fold].index
    
    train_df = train[train['fold'] != fold].reset_index(drop=True)
    valid_df = train[train['fold'] == fold].reset_index(drop=True)
    
    train_images = train_df.image_id.values.tolist()
    train_images = [os.path.join(training_data_path, i) for i in train_images]
    train_targets = train_df.label.values

    valid_images = valid_df.image_id.values.tolist()
    valid_images = [os.path.join(training_data_path, i) for i in valid_images]
    valid_targets = valid_df.label.values               
        
    train_dataset = Cassava_Train_DS(train_images, train_targets, resize = resize, augmentations = train_aug)
    valid_dataset = Cassava_Train_DS(valid_images, valid_targets, resize = resize, augmentations = valid_aug)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
    model = CassavaModel()
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.CrossEntropyLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
   
    oof = np.zeros((len(train), 5))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_fn, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        score = accuracy_score(valid_targets, valid_preds.argmax(1))
        print(f"fold: {fold} Accuracy: {score}")
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"{MODEL_PATH}{fold}_{seed}.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    testdataset = Cassava_Test_DS(test_images, resize = resize , augmentations = train_aug)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = CassavaModel()
    
    model.load_state_dict(torch.load(f"{MODEL_PATH}{fold}_{seed}.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_images), 5))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions


In [56]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train_df), 5))
    predictions = np.zeros((len(test_images), 5))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [57]:
# Averaging on multiple SEEDS

SEED = [0,] #<-- Update
oof = np.zeros((len(train_df), 5))
predictions = np.zeros((len(test_images), 5))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)
cols_ = ['pred1','pred2','pred3','pred4','pred5']
for i in cols_:
    train_df[i] = 0  
    test[i] = 0
train_df[cols_] = oof
test[cols_] = predictions

FOLD: 0, EPOCH: 0, train_loss: 1.117829452162591
(4280, 5)
FOLD: 0, EPOCH: 0, valid_loss: 1.7112277514898955
fold: 0 Accuracy: 0.47289719626168225
FOLD: 0, EPOCH: 1, train_loss: 1.082669525057356
(4280, 5)
FOLD: 0, EPOCH: 1, valid_loss: 1.0714383681318653
fold: 0 Accuracy: 0.6238317757009346
FOLD: 0, EPOCH: 2, train_loss: 0.9800557067461103
(4280, 5)
FOLD: 0, EPOCH: 2, valid_loss: 1.0446288158644492
fold: 0 Accuracy: 0.647196261682243
FOLD: 0, EPOCH: 3, train_loss: 0.9111759506653403
(4280, 5)
FOLD: 0, EPOCH: 3, valid_loss: 0.8499583862165907
fold: 0 Accuracy: 0.6785046728971963
FOLD: 0, EPOCH: 4, train_loss: 0.8522492832112535
(4280, 5)
FOLD: 0, EPOCH: 4, valid_loss: 0.8663559120092819
fold: 0 Accuracy: 0.6754672897196262
FOLD: 0, EPOCH: 5, train_loss: 0.8104697702086975
(4280, 5)
FOLD: 0, EPOCH: 5, valid_loss: 0.8092832556411401
fold: 0 Accuracy: 0.7144859813084112
FOLD: 0, EPOCH: 6, train_loss: 0.7608155907871568
(4280, 5)
FOLD: 0, EPOCH: 6, valid_loss: 0.7228878029246828
fold: 0 Ac

In [81]:
predictions

array([[0.05365807, 0.09514696, 0.75128658, 0.77435602, 0.39148922]])

In [74]:
oof.argmax(1)

array([4, 3, 3, ..., 2, 3, 1], dtype=int64)

In [65]:
sample_submission = pd.read_csv('D:\\Dataset\\Cassava Competiton\\sample_submission.csv')

In [71]:
predicted = pd.DataFrame({'image_id' : test_images_sub, 'label' : predictions.argmax(1)})

In [80]:
test

,image_id,Prediction,pred1,pred2,pred3,pred4,pred5
0,2216849948.jpg,-1,0.053658,0.095147,0.751287,0.774356,0.391489
